# Agents and Tools


### Install Python Prerequisist

As always, let's start by installing the Python Libraries we neeed

In [ ]:
!pip install -U llama-stack-client==0.2.5 dotenv

### Define the LLamastack server and Model

Let's point our variables to our Llamastack server and chose our desired model: 

In [ ]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for our lab, we will just define our variables manualy here:
## if you are running from jupyter-nodes server or doing a lab with RHDP:
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'
## If you are running from a jupyter-notes container:
os.environ['LLAMA_STACK_SERVER'] = 'http://host.containers.internal:8321'
os.environ['LLAMA_STACK_MODEL'] = 'meta-llama/Llama-3.2-3B-Instruct'

> **Note:**
>When running this code in a regular Python application, we would usually like to read environment variables from an `.env` file, for our needs in this lab, we will hard code these in this cell, to make things more clear
>

### Initialize the *Client* 
As a first step, let's define our client, provide it our Llama-Stack Server location and select the model we would like to work with, later, we will see that pointing this to a different location (Llama-Stack Serve) is all we would need to do to move to a production environment.

In [ ]:
tavily_search_api_key='tvly-BROKENdBROKENf89fUfZMUSIe'

provider_data = {"tavily_search_api_key": tavily_search_api_key}

In [ ]:
from llama_stack_client import LlamaStackClient

LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")
LLAMA_STACK_MODEL=os.getenv("LLAMA_STACK_MODEL")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
    provider_data=provider_data
)
# List available models
models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")


In [ ]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client, 
    model='meta-llama/Llama-3.2-3B-Instruct',
    #model='meta-llama/Llama-3.1-8B-Instruct',
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    #sampling_params=sampling_params
)



In [ ]:

from llama_stack_client.lib.agents.event_logger import EventLogger
stream=True
user_prompts = [
       "search the web for the latest in OpenShift?",
       "search for the current weather in boston",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

Wow look at that, notice that the agent is kinda silly.. if you try a nother word, it might fail, thats more a symptom of a small LLM than the agent... 

Now lets try REact agents

In [ ]:
The next cell is meant to halucinate

In [ ]:
user_prompts = [
    "Search for any weather-related risks in my area that could disrupt network connectivity or system availability?",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session1")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

In [ ]:
!pip install geocoder


In [ ]:
from llama_stack_client.lib.agents.client_tool import client_tool

@client_tool
def get_location(query: str = "location"):
    """
    Provides the user's location upon request.

    This function uses the geocoder library to determine the user's location
    based on their IP address.  It returns the city, state, and country.

    :param query:  The query from the user.  Defaults to "location".
    :type query: str
    :return:  Information about the user's current location.
    :rtype: str

    Example:
        >>> get_location("where am i")
        "Your current location is: Some City, Some State, Some Country"
    """
    import geocoder
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"
#test_geo = get_location("where am i")
#print(test_geo)


In [ ]:

agent = Agent(
    client, 
    model='meta-llama/Llama-3.2-3B-Instruct',
    #model='meta-llama/Llama-3.1-8B-Instruct',
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    #sampling_params=sampling_params
)

In [ ]:
stream=True
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",
    "Search for the current weather near me"
]
session_id = agent.create_session("prompt-chaining-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

The above doesn't work properly yet.

## ReActAGent

In [ ]:
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

In [ ]:
agent = ReActAgent(
            client=client,
            model='meta-llama/Llama-3.2-3B-Instruct',
            #model='meta-llama/Llama-3.1-8B-Instruct',
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            #sampling_params=sampling_params,
        )
user_prompts = [
    "search for any weather-related risks near my location"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 